# Decision Tree Implementation

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import copy
import sys

## Split on a given feature

Function **get_splits** gets splits for different values of the $i^{th}$ feature. For example, for a feature with values [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4.75], the unique values are [1, 2, 3, 4, 4.75]. Therefore, the splits chosen are [1.5, 2.5, 3.5, 4.375].

In [2]:
def get_splits(X, i):
    unique_vals = np.unique(X[:, i])
    splits = np.zeros(unique_vals.shape[0] - 1)
    
    for i in range(len(splits)):
        # Choose a value in between the two unique vals
        splits[i] = (unique_vals[i] + unique_vals[i + 1]) / 2
        
    return splits

get_splits(np.array([[1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4.75]]).T, 0)

array([1.5  , 2.5  , 3.5  , 4.375])

## Entropy

Entropy is given by $E = -\sum_{i}{p_{i}(x) \log_{2}(p_{i}(x))}$

In [3]:
def get_entropy(target):
    _, counts = np.unique(target, return_counts=True)
    
    probs = counts / target.shape[0]
    
    return -np.sum(probs * np.log2(probs), axis=0)

The entropy is 0 for a pure split.

In [4]:
get_entropy(np.array([1, 1, 1, 1, 1, 1, 1, 1]))

-0.0

The entropy is 1 for a split with equal number of examples from each of the classes.

In [5]:
get_entropy(np.array([1, 1, 1, 1, 0, 0, 0, 0]))

1.0

The following function computes the loss of entropy / gain of information. At any depth in the decision tree, our objective shall be to find a split which leads to best information gain. The split is done in a greedy fashion.

In [6]:
def get_information_gain(y, left_y, right_y):
    return get_entropy(y) - get_entropy(left_y) - get_entropy(right_y)

## Get the Best Split (Greedy Algorithm)

**get_best_split** function runs a greedy algorithm to choose a feature and a split on it that leads to maximum information gain.

In [7]:
def get_best_split(X, y):
    
    max_info_gain = -sys.maxsize
    max_left_y = None
    max_right_y = None
    max_left_X = None
    max_right_X = None
    
    for i in range(X.shape[1]):
        splits = get_splits(X, i)
        
        for split in splits:
            left_indices = X[:, i] < split
            right_indices = X[:, i] > split

            left_y = y[left_indices]
            right_y = y[right_indices]

            info_gain = get_information_gain(y, left_y, right_y)

            if info_gain > max_info_gain:
                max_info_gain = info_gain
                max_left_y = left_y
                max_right_y = right_y
                max_left_X = X[left_indices, :]
                max_right_X = X[right_indices, :]
                max_split = split
                max_feature = i
    
    return max_left_X, max_left_y, max_right_X, max_right_y, max_split, max_feature, max_info_gain

Now let's test our function on an input.

In [8]:
X = np.array([[0, 0, 1, 1, 1, 2, 2, 2]]).T
y = np.array([0, 0, 1, 1, 1, 2, 2, 1])
splits = np.array([.5, 1.5])
get_best_split(X, y)

(array([[0],
        [0]]), array([0, 0]), array([[1],
        [1],
        [1],
        [2],
        [2],
        [2]]), array([1, 1, 1, 2, 2, 1]), 0.5, 0, 0.5817041659455104)

In [9]:
def get_probs(y, n_classes):
    # Get counts and infer the probability (also infer counts for classes not present in this sample)
    return np.bincount(y, minlength=n_classes) / len(y)

## Recursive Function to Construct Decision Tree

At each depth we:
1. Check if we are splitting on a sample less than *min_sample_split*. If yes, then stop recurring.
2. Check if max depth has been exceeded
3. Find the best possible split using the greedy function **get_best_split**
4. Check if the information gain returned by the function is less than our *min_info_gain* parameter, if yes, then stop recurring.
5. Set the feature to split, the split value and the corresponding information gain into current node. Then we recur for left and right subtrees.

In [10]:
def construct_tree(X, y, min_info_gain, max_depth, min_sample_split, tree, n_classes):
    
    if X.shape[0] < min_sample_split or max_depth < 0:
        tree['probs'] = get_probs(y, n_classes)
        return
    
    left_X,\
    left_y,\
    right_X,\
    right_y,\
    split,\
    feature,\
    info_gain = get_best_split(X, y)

    if info_gain < min_info_gain:
        tree['probs'] = get_probs(y, n_classes)
        return
    
    tree['feature'] = feature
    tree['split'] = split
    tree['info_gain'] = info_gain
    tree['left'] = {}
    tree['right'] = {}
    
    construct_tree(left_X, left_y, min_info_gain, max_depth - 1, min_sample_split, tree['left'], n_classes)
    construct_tree(right_X, right_y, min_info_gain, max_depth - 1, min_sample_split, tree['right'], n_classes)

## Prediction function

For a given row, we can recursively traverse the decision tree based on the feature and split chosen at each node.

In [11]:
def predict_row(x_row, tree):
    
    # If this is not a leaf node
    if 'split' not in tree:
        return copy.deepcopy(tree['probs'])
    
    if x_row[tree['feature']] < tree['split']:
        return predict_row(x_row, tree['left'])
    
    return predict_row(x_row, tree['right'])

## Decision Tree Class Definition

In [13]:
class DecisionTree:
    def __init__(self, max_depth=3, min_sample_split=10, min_info_gain=1e-7):
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.min_info_gain = min_info_gain
        self.tree = {}
        
    def fit(self, X, y):
        self.n_classes = np.unique(y).shape[0]
        construct_tree(X, y, self.min_info_gain, self.max_depth, self.min_sample_split, self.tree, self.n_classes)
    
    # Predicts the probability
    def predict_proba(self, X):
        
        res = np.zeros((X.shape[0], self.n_classes))
        
        for i in range(X.shape[0]):
            x_row = X[i, :]
            
            probs = predict_row(x_row, self.tree)
        
            res[i, :] = probs
        
        return res
    
    # Prediction using probabilities given by the decision tree
    def predict(self, X):
        proba = self.predict_proba(X)
        
        return np.argmax(proba, axis=1)

This completes the code for our custom decision tree. Now let's try it out on the toy IRIS dataset which comes inbuilt with sklearn.

In [14]:
iris_data = load_iris()
X = iris_data['data'] # Features
y = iris_data['target'] # Target classes

In [15]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [16]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [17]:
model = DecisionTree()

Test size of 33 percent was chosen.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Fitting the decision tree onto X and y.

In [19]:
model.fit(X_train, y_train)

In [20]:
y_pred = model.predict(X_test)

Our model achieves an accuracy of 98 percent on the test set.

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.98